In [ ]:
import time
import pyrootutils
import pytorch_lightning as pl
import wandb

# set pythonpath and working directory to folder containing .project-root file
root = pyrootutils.setup_root(".", indicator=".project-root", pythonpath=True, cwd=True)

from src.data.face_age_datamodule import FaceAgeDataModule
from src.models.face_age_module import FaceAgeModule


def main():
    
    # execute for 3 seeds
    for i in range(3):
        # set seed for reproducibility
        pl.seed_everything(i)

        data_dir = root / "data"
        log_dir = root / "logs" / time.strftime("%Y-%m-%d_%H-%M-%S", time.localtime())

        use_wandb = True
        age_norm_value = 80
        use_augmented_dataset = False
        # loss_fn = "MSELoss"
        loss_fn = "SmoothL1Loss"

        net = "EffNet_224x224"
        img_size = (224, 224)
        imagenet_normalization = True
        exp_name = f"EffNet+{img_size}+augmented={use_augmented_dataset}+{loss_fn}"

        datamodule = FaceAgeDataModule(
            data_dir=data_dir,
            img_size=img_size,
            imagenet_normalization=imagenet_normalization,
            use_augmented_dataset=use_augmented_dataset,
            normalize_age_by=age_norm_value,
            num_workers=0,
            batch_size=32,
            pin_memory=True,
        )

        model = FaceAgeModule(net=net, rescale_age_by=age_norm_value, loss_fn=loss_fn)

        callbacks = []
        loggers = []

        # this controls how checkpoints are saved
        callbacks.append(
            pl.callbacks.ModelCheckpoint(
                monitor="val/loss",
                dirpath=log_dir / "checkpoints",
                save_top_k=1,  # save the best checkpoint
                save_last=True,  # additionally the save the last checkpoint
                mode="min",
                save_weights_only=True,
                filename="best-checkpoint",
            )
        )

        # this configurates optional weights&biases logger
        if use_wandb:
            loggers.append(
                pl.loggers.WandbLogger(
                    project="face-age",
                    save_dir=log_dir,
                    name=exp_name,
                    group=exp_name,
                )
            )

        # trainer setup
        trainer = pl.Trainer(
            accelerator="gpu",
            default_root_dir=log_dir,
            callbacks=callbacks,
            logger=loggers,
            max_epochs=10,
            val_check_interval=0.2,  # frequency of validation epoch
        )

        # train
        trainer.fit(model=model, datamodule=datamodule)

        # test
        trainer.test(model=model, datamodule=datamodule, ckpt_path="best")

        if use_wandb:
            wandb.finish()

In [ ]:
main()